**DOG BREED CLASSIFICATION**

This is a dog breed classification script which makes use of Transfer learning. A pre-trained Inception V3 is used as a feature extractor and the fully connected layers have been swapped with the custom dog breed classes.

The Stanford Dog Dataset is used, which contains 120 breeds of dog and has more than 20000 images.

Dataset Reference
Primary:
  Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2011.  [pdf]  [poster]  [BibTex]

Secondary:
  J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database. IEEE Computer Vision and Pattern Recognition (CVPR), 2009.  [pdf]  [BibTex]

In [1]:
!pip install -q tensorflow_hub

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


All the necessary imports.

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras.layers as layers
import os

W0528 21:46:48.592976 16236 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
tf.VERSION


'1.13.1'

The compressed image file is downloaded from 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar ' and decompressed into keras' cache folder.

The get_file() function of the keras.utils returns the file path which is stored in data_root.

In [4]:
data_root = tf.keras.utils.get_file('Images' , 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar' , untar=True)

Confirming the directory.

In [5]:
!ls /root/.keras/datasets/



'ls' is not recognized as an internal or external command,
operable program or batch file.


The bottom pre-trained weight vectors of Inception model is downloaded from tfhub.
Link:- "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3 "

In [6]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3"

Creating the module and getting the image size on which the Inception model was trained.

In [7]:
def feature_extractor(x):
  feature_extractor_module = hub.Module(feature_extractor_url)
  return feature_extractor_module(x)

IMAGE_SIZE = hub.get_expected_image_size(hub.Module(feature_extractor_url))

IMAGE_SIZE

Instructions for updating:
Colocations handled automatically by placer.


W0528 21:47:51.282402 16236 deprecation.py:323] From C:\Users\ajax9\Anaconda3\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


[299, 299]

Image generator to pre-process the images and scale them so that the data is in range [0, 1]

In [8]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

All the images in the diectory are rescaled to the image size accepted by Inception model.
The resulting object is an iterator which returns image and label batch pairs.

In [9]:
image_data = image_generator.flow_from_directory(str(data_root) , target_size = IMAGE_SIZE)

Found 16835 images belonging to 99 classes.


In [10]:
for img , lab in image_data:
  print(img.shape)
  print(lab.shape)
  break

(32, 299, 299, 3)
(32, 99)


Wrapping the layers in a keras layer

In [11]:
feature_extractor_layer = layers.Lambda(feature_extractor , input_shape  = IMAGE_SIZE + [3])

Freezing the bottom layers so that they arent trained again and only the new custom layer is trained.

In [12]:
feature_extractor_layer.trainable = False

Building the custom model with feature extracting layer of Inception on top of custom layer with softmax activation.

In [13]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(image_data.num_classes , activation = "softmax")
])

model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 21:48:16.831165 16236 saver.py:1483] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 99)                202851    
Total params: 202,851
Trainable params: 202,851
Non-trainable params: 0
_________________________________________________________________


Initializing the tfHub module.

In [14]:
import tensorflow.keras.backend as K
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)

Confirming the resultant shape

In [15]:
result = model.predict(img)
result.shape

(32, 99)

Configuring the training process. 


*   Using Adam optimizer to optimize the output to maximum accuracy
*   Loss as 'categorical crossentropy' as, multiclass classifiction
*   Metrics as accuracy



In [16]:
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

To visualize the training progress during every epoch,  a custom callback to log the loss and accuract of each batch

In [17]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    
  def on_batch_end(self , batch , logs = None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    

In [18]:
steps = image_data.samples//image_data.batch_size
steps

526

Training the model for 5 epochs.

In [ ]:
batch_stats = CollectBatchStats()

model.fit(
    (items for items in image_data),
    epochs = 5,
    steps_per_epoch = steps,
    callbacks = [batch_stats]
)

Instructions for updating:
Use tf.cast instead.


W0528 21:48:36.506582 16236 deprecation.py:323] From C:\Users\ajax9\Anaconda3\lib\site-packages\tensorflow\python\ops\math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/5
526/526 [==============================] - 3160s 6s/step - loss: 0.6953 - acc: 0.8397
Epoch 2/5
526/526 [==============================] - 3020s 6s/step - loss: 0.2549 - acc: 0.9168
Epoch 3/5
202/526 [==========>...................] - ETA: 14:20:44 - loss: 0.1891 - acc: 0.9386

Plotting the loss and accuracy in every training step.

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats.batch_losses)

plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats.batch_acc)

Ordering the class names.

In [ ]:
import numpy as np

label_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
label_names = np.array([key.title() for key, value in label_names])
label_names.shape


Formatting the string to get only the breed name, omitting everything efore the ' - '.

In [ ]:
def formatString(x):
  i = x.find('-')
  return x[i+1:]
    

Predicting breeds of input images.

In [ ]:
from keras.preprocessing import image
image1 = image.load_img('thumbnail-1477089693232.jpg' , target_size=IMAGE_SIZE)
img_tensor = image.img_to_array(image1)
img_tensor = np.expand_dims(img_tensor , axis=0)
img_tensor /= 255.

result_batch = model.predict(img_tensor)

labels_batch = label_names[np.argmax(result_batch, axis=-1)]
unformatted_prediction = labels_batch[0]
prediction = formatString(unformatted_prediction)
prediction

In [ ]:
plt.figure(figsize=(5,5))

#plt.subplot(6,5,1)
plt.imshow(img_tensor[0])
plt.title(formatString(labels_batch[0]))
plt.axis('off')
_ = plt.suptitle("Model predictions")